In [1]:
import sys, os, math
sys.path.append('/mnt/volume/shared/anna/CMSCB-SCEMILA/SCEMILA/ml_pipeline')
import statistics
import numpy as np
import pandas as pd
import pickle as pkl
import os.path
from tqdm import tqdm
import torch
import torch.nn.functional as F
from dataset_mixed import *  # dataset
from model import *  # actual MIL model
from sklearn import metrics as metrics
import csv

import umap

import label_converter

# import functions instead of having them in here to keep the notebook *much* shorter
sys.path.append('functions')


/tmp/ipykernel_440638/820541183.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
CLASSES = ['control', 'RUNX1_RUNX1T1', 'NPM1', 'CBFB_MYH11', 'PML_RARA']
SOURCE_FOLDER = "/mnt/volume/shared/test_data"
patients = {}
with open(os.path.join(SOURCE_FOLDER,'metadata.csv'), newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        # print(line)
        patients[line[1]] = [os.path.join(SOURCE_FOLDER,
                                         "data",
                                         line[4],
                                         line[1],
                                         "fnl34_bn_features_layer_7.npy"), line[4]]

In [3]:
def classwise_area_under_pr(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        y_pred_class = predictions[:, class_index]
        y_true_class = ground_truth[:, class_index]

        # Compute precision-recall curve
        precision, recall, _ = metrics.precision_recall_curve(y_true_class, y_pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = metrics.auc(recall, precision)

    return classwise_auc

def classwise_area_under_roc(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        pred_class = (predictions == class_index).astype(int)
        gt_class = (ground_truth == class_index).astype(int)

        # Compute precision-recall curve
        score = -1
        try:
            score = metrics.roc_auc_score(gt_class, pred_class)
        except:
            print("error")
        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def classwise_acc(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        y_pred_class = predictions[:, class_index]
        y_true_class = ground_truth[:, class_index]

        # Compute precision-recall curve
        score = metrics.accuracy(y_true_class, y_pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def classwise_f1(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        pred_class = (predictions == class_index).astype(int)
        gt_class = (ground_truth == class_index).astype(int)

        # Compute precision-recall curve
        score = metrics.f1_score(gt_class, pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def calculate_metrics(pred,gt, classes):
    pred = np.array(pred).squeeze()
    gt = np.array(gt)
    pred0 = np.argmax(pred, axis=1)
    gt0 = np.argmax(gt, axis=1)
    acc = metrics.accuracy_score(gt0, pred0)
    f1 = metrics.f1_score(gt0, pred0, average="micro")
    pr = metrics.precision_score(gt0, pred0, average="micro")
    re = metrics.recall_score(gt0, pred0, average="micro")
    matrix = metrics.confusion_matrix(gt0, pred0)
    c_acc = matrix.diagonal()/matrix.sum(axis=1)
    print(metrics.classification_report(gt0, pred0, target_names=classes))
    c_f1 = classwise_f1(pred0,gt0, 5)

    auc_pr = classwise_area_under_pr(pred, gt, 5)
    auc_roc = classwise_area_under_roc(pred0, gt0, 5)

    # print("Area under the PR curve for each class:", auc_pr)

    return acc, f1, pr, re, auc_pr, auc_roc, c_acc, c_f1

def correct_order_classes(converter,auc):
    out = np.zeros(len(CLASSES))

    for i in range(len(CLASSES)):
        out[i] = auc[converter[CLASSES[i]]]

    return out

In [4]:
def get_preds(patients, model, class_converter):
    pred = []
    gt = []
    for p in patients.keys():
            path, lbl_name = patients[p]
            lbl = np.zeros(5)
            lbl[class_converter[lbl_name]] = 1

            bag = np.load(path)

            bag = torch.tensor(bag)
            bag = torch.unsqueeze(bag,0)
            p, _, _, _ = model(bag)
            pred.append(F.softmax(p,dim=1).cpu().detach().numpy())
            gt.append(lbl)
    return pred, gt


In [7]:
class_converter = {}
with open("/mnt/volume/shared/class_conversion.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        class_converter[line[1]] = int(line[0])
paths = {"/mnt/volume/shared/all_results/exp0/result_folder_1": "exp0_rep0",
         "/mnt/volume/shared/all_results/exp0_try0/result_folder": "exp0_rep1",
         "/mnt/volume/shared/all_results/exp0_try1/result_folder": "exp0_rep2",
         "/mnt/volume/shared/all_results/exp1/result_folder_1": "exp1",
         "/mnt/volume/shared/all_results/exp2/result_folder_1": "exp2",
         "/mnt/volume/shared/new_results/experiment_3_seed1/result_folder": "exp3_seed1",
         "/mnt/volume/shared/new_results/experiment_3_seed20/result_folder": "exp3_seed20",
         "/mnt/volume/shared/new_results/experiment_3_seed42/result_folder": "exp3_seed42",
         "/mnt/volume/shared/new_results/mixed_seed1/10_percent/result_folder": "mixed_seed1_10percent",
         "/mnt/volume/shared/new_results/mixed_seed20/10_percent/result_folder": "mixed_seed20_10percent",
         "/mnt/volume/shared/new_results/mixed_seed42/10_percent/result_folder": "mixed_seed42_10percent",
         "/mnt/volume/shared/new_results/mixed_seed1/20_percent/result_folder": "mixed_seed1_20percent",
         "/mnt/volume/shared/new_results/mixed_seed20/20_percent/result_folder": "mixed_seed20_20percent",
         "/mnt/volume/shared/new_results/mixed_seed42/20_percent/result_folder": "mixed_seed42_20percent",
         "/mnt/volume/shared/new_results/mixed_seed1/30_percent/result_folder": "mixed_seed1_30percent",
         "/mnt/volume/shared/new_results/mixed_seed20/30_percent/result_folder": "mixed_seed20_30percent",
         "/mnt/volume/shared/new_results/mixed_seed42/30_percent/result_folder": "mixed_seed42_30percent",
         "/mnt/volume/shared/new_results/mixed_seed1/50_percent/result_folder": "mixed_seed1_50percent",
         "/mnt/volume/shared/new_results/mixed_seed20/50_percent/result_folder": "mixed_seed20_50percent",
         "/mnt/volume/shared/new_results/mixed_seed42/50_percent/result_folder": "mixed_seed42_50percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_10_percent/result_folder": "uncertain_seed1_max_10percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_10_percent/result_folder": "uncertain_seed20_max_10percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_10_percent/result_folder": "uncertain_seed41_max_10percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_20_percent/result_folder": "uncertain_seed1_max_20percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_20_percent/result_folder": "uncertain_seed20_max_20percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_20_percent/result_folder": "uncertain_seed42_max_20percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_30_percent/result_folder": "uncertain_seed1_max_30percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_30_percent/result_folder": "uncertain_seed20_max_30percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_30_percent/result_folder": "uncertain_seed42_max_30percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_50_percent/result_folder": "uncertain_seed1_max_50percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_50_percent/result_folder": "uncertain_seed20_max_50percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_50_percent/result_folder": "uncertain_seed42_max_50percent",
        }
inv_class = {v: k for k, v in class_converter.items()}
classes = []
for i in range(5):
    classes.append(inv_class[i])

In [8]:
df = pd.DataFrame(columns=['path','acc','f1','pr', 're', 'auc_pr','auc_roc','c_acc','c_f1'])
for path in paths.keys():
    model = torch.load(os.path.join(path,"state_dictmodel.pt"),map_location="cpu")
    pred, gt = get_preds(patients, model, class_converter)
    acc, f1, pr, re, auc_pr, auc_roc, c_acc, c_f1 = calculate_metrics(pred, gt, classes)
    df.loc[len(df.index)] = [paths[path], '%.4f' % acc, '%.4f' % f1, '%.4f' % pr, '%.4f' % re, ['%.2f' % elem for elem in auc_pr] , ['%.2f' % elem for elem in auc_roc], ['%.2f' % elem for elem in c_acc], ['%.2f' % elem for elem in c_f1]]

               precision    recall  f1-score   support

RUNX1_RUNX1T1       1.00      0.67      0.80         6
      control       1.00      1.00      1.00        12
         NPM1       0.86      0.86      0.86         7
     PML_RARA       1.00      1.00      1.00         5
   CBFB_MYH11       0.67      0.86      0.75         7

     accuracy                           0.89        37
    macro avg       0.90      0.88      0.88        37
 weighted avg       0.91      0.89      0.89        37

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.80      0.67      0.73         6
      control       1.00      1.00      1.00        12
         NPM1       0.86      0.86      0.86         7
     PML_RARA       1.00      0.80      0.89         5
   CBFB_MYH11       0.67      0.86      0.75         7

     accuracy                           0.86        37
    macro avg       0.86      0.84      0.84        37
 weighted avg       0.88      0.86      0.87        37

    

/mnt/volume/shared/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/volume/shared/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/volume/shared/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.00      0.00      0.00         6
      control       0.32      1.00      0.49        12
         NPM1       0.00      0.00      0.00         7
     PML_RARA       0.00      0.00      0.00         5
   CBFB_MYH11       0.00      0.00      0.00         7

     accuracy                           0.32        37
    macro avg       0.06      0.20      0.10        37
 weighted avg       0.11      0.32      0.16        37

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.50      0.50      0.50         6
      control       1.00      1.00      1.00        12
         NPM1       1.00      0.57      0.73         7
     PML_RARA       1.00      0.20      0.33         5
   CBFB_MYH11       0.43      0.86      0.57         7

     accuracy                           0.70        37
    macro avg       0.79      0.63      0.63        37
 weighted avg       0.81      0.70      0.70        37

    

In [9]:
df

,path,acc,f1,pr,re,auc_pr,auc_roc,c_acc,c_f1
0,exp0_rep0,0.8919,0.8919,0.8919,0.8919,"[0.92, 1.00, 0.89, 1.00, 0.82]","[0.83, 1.00, 0.91, 1.00, 0.88]","[0.67, 1.00, 0.86, 1.00, 0.86]","[0.80, 1.00, 0.86, 1.00, 0.75]"
1,exp0_rep1,0.8649,0.8649,0.8649,0.8649,"[0.87, 1.00, 0.87, 1.00, 0.82]","[0.82, 1.00, 0.91, 0.90, 0.88]","[0.67, 1.00, 0.86, 0.80, 0.86]","[0.73, 1.00, 0.86, 0.89, 0.75]"
2,exp0_rep2,0.8378,0.8378,0.8378,0.8378,"[0.86, 1.00, 0.87, 1.00, 0.82]","[0.73, 1.00, 0.90, 0.90, 0.88]","[0.50, 1.00, 0.86, 0.80, 0.86]","[0.60, 1.00, 0.80, 0.89, 0.75]"
3,exp1,0.3243,0.3243,0.3243,0.3243,"[0.18, 0.50, 0.22, 0.15, 0.19]","[0.50, 0.50, 0.50, 0.50, 0.50]","[0.00, 1.00, 0.00, 0.00, 0.00]","[0.00, 0.49, 0.00, 0.00, 0.00]"
4,exp2,0.7027,0.7027,0.7027,0.7027,"[0.60, 1.00, 0.83, 0.83, 0.71]","[0.70, 1.00, 0.79, 0.60, 0.80]","[0.50, 1.00, 0.57, 0.20, 0.86]","[0.50, 1.00, 0.73, 0.33, 0.57]"
5,exp3_seed1,0.7838,0.7838,0.7838,0.7838,"[0.50, 1.00, 0.72, 1.00, 0.93]","[0.70, 1.00, 0.72, 1.00, 0.84]","[0.50, 1.00, 0.57, 1.00, 0.71]","[0.50, 1.00, 0.53, 1.00, 0.77]"
6,exp3_seed20,0.7027,0.7027,0.7027,0.7027,"[0.50, 1.00, 0.62, 1.00, 0.81]","[0.47, 1.00, 0.72, 0.90, 0.84]","[0.00, 1.00, 0.71, 0.80, 0.71]","[0.00, 1.00, 0.50, 0.89, 0.77]"
7,exp3_seed42,0.7838,0.7838,0.7838,0.7838,"[0.39, 1.00, 0.66, 1.00, 0.86]","[0.70, 0.98, 0.75, 0.90, 0.90]","[0.50, 1.00, 0.57, 0.80, 0.86]","[0.50, 0.96, 0.62, 0.89, 0.80]"
8,mixed_seed1_10percent,0.8919,0.8919,0.8919,0.8919,"[0.92, 1.00, 0.90, 1.00, 0.97]","[0.90, 1.00, 0.91, 0.90, 0.90]","[0.83, 1.00, 0.86, 0.80, 0.86]","[0.83, 1.00, 0.86, 0.89, 0.80]"
9,mixed_seed20_10percent,0.8919,0.8919,0.8919,0.8919,"[0.96, 1.00, 0.91, 1.00, 0.76]","[0.90, 1.00, 0.91, 0.90, 0.90]","[0.83, 1.00, 0.86, 0.80, 0.86]","[0.83, 1.00, 0.86, 0.89, 0.80]"


In [10]:
df.to_csv("./results/raw_res.csv")

In [11]:
def get_res_df(paths):
    df_exp3 = pd.DataFrame(columns=['path','acc','f1','pr', 're', 'auc_pr','auc_roc','c_acc','c_f1'])
    for path in paths.keys():
        model = torch.load(os.path.join(path,"state_dictmodel.pt"),map_location="cpu")
        pred, gt = get_preds(patients, model, class_converter)
        acc, f1, pr, re, auc_pr, auc_roc, c_acc, c_f1 = calculate_metrics(pred, gt, classes)
        df_exp3.loc[len(df_exp3.index)] = [paths[path], acc, f1, pr, re, auc_pr , auc_roc, c_acc, c_f1]
    return df_exp3

In [12]:
def get_mean_std(df_exp3):
    df_exp3[['auc_pr_RUNX1_RUNX1T1','auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11']] = pd.DataFrame(df_exp3.auc_pr.tolist(), index= df_exp3.index)
    df_exp3.drop(['auc_pr'], axis=1, inplace=True)
    df_exp3[['auc_pr_RUNX1_RUNX1T1','auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11']] = df_exp3[['auc_pr_RUNX1_RUNX1T1','auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11']].apply(pd.to_numeric)
    df_exp3[['auc_roc_RUNX1_RUNX1T1','auc_roc_control', 'auc_roc_NPM1', 'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11']] = pd.DataFrame(df_exp3.auc_roc.tolist(), index= df_exp3.index)
    df_exp3.drop(['auc_roc'], axis=1, inplace=True)
    df_exp3[['auc_roc_RUNX1_RUNX1T1','auc_roc_control', 'auc_roc_NPM1', 'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11']] = df_exp3[['auc_roc_RUNX1_RUNX1T1','auc_roc_control', 'auc_roc_NPM1', 'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11']].apply(pd.to_numeric)
    df_exp3[['c_acc_RUNX1_RUNX1T1','c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11']] = pd.DataFrame(df_exp3.c_acc.tolist(), index= df_exp3.index)
    df_exp3.drop(['c_acc'], axis=1, inplace=True)
    df_exp3[['c_acc_RUNX1_RUNX1T1','c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11']] = df_exp3[['c_acc_RUNX1_RUNX1T1','c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11']].apply(pd.to_numeric)
    df_exp3[['c_f1_RUNX1_RUNX1T1','c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA', 'c_f1_CBFB_MYH11']] = pd.DataFrame(df_exp3.c_f1.tolist(), index= df_exp3.index)
    df_exp3.drop(['c_f1'], axis=1, inplace=True)
    df_exp3[['c_f1_RUNX1_RUNX1T1','c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA', 'c_f1_CBFB_MYH11']] = df_exp3[['c_f1_RUNX1_RUNX1T1','c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA', 'c_f1_CBFB_MYH11']].apply(pd.to_numeric)
    out = pd.concat([df_exp3, df_exp3.describe().loc[["mean", "std"]]])
    return out

In [14]:
paths_exp0 = {"/mnt/volume/shared/all_results/exp0/result_folder_1": "exp0_rep0",
              "/mnt/volume/shared/all_results/exp0_try0/result_folder": "exp0_rep1",
              "/mnt/volume/shared/all_results/exp0_try1/result_folder": "exp0_rep2",
             }
paths_exp3 = {"/mnt/volume/shared/new_results/experiment_3_seed1/result_folder": "exp3_seed1",
         "/mnt/volume/shared/new_results/experiment_3_seed20/result_folder": "exp3_seed20",
         "/mnt/volume/shared/new_results/experiment_3_seed42/result_folder": "exp3_seed42"}
paths_mix10 = {"/mnt/volume/shared/new_results/mixed_seed1/10_percent/result_folder": "mixed_seed1_10percent",
         "/mnt/volume/shared/new_results/mixed_seed20/10_percent/result_folder": "mixed_seed20_10percent",
         "/mnt/volume/shared/new_results/mixed_seed42/10_percent/result_folder": "mixed_seed42_10percent"}
paths_mix20 = {"/mnt/volume/shared/new_results/mixed_seed1/20_percent/result_folder": "mixed_seed1_20percent",
         "/mnt/volume/shared/new_results/mixed_seed20/20_percent/result_folder": "mixed_seed20_20percent",
         "/mnt/volume/shared/new_results/mixed_seed42/20_percent/result_folder": "mixed_seed42_20percent"}
paths_mix30 = {"/mnt/volume/shared/new_results/mixed_seed1/30_percent/result_folder": "mixed_seed1_30percent",
         "/mnt/volume/shared/new_results/mixed_seed20/30_percent/result_folder": "mixed_seed20_30percent",
         "/mnt/volume/shared/new_results/mixed_seed42/30_percent/result_folder": "mixed_seed42_30percent"}
paths_mix50 = {"/mnt/volume/shared/new_results/mixed_seed1/50_percent/result_folder": "mixed_seed1_50percent",
         "/mnt/volume/shared/new_results/mixed_seed20/50_percent/result_folder": "mixed_seed20_50percent",
         "/mnt/volume/shared/new_results/mixed_seed42/50_percent/result_folder": "mixed_seed42_50percent"}
paths_uncertain_max10 = {"/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_10_percent/result_folder": "uncertain_seed1_max_10percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_10_percent/result_folder": "uncertain_seed20_max_10percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_10_percent/result_folder": "uncertain_seed41_max_10percent",}
paths_uncertain_max20 = {"/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_20_percent/result_folder": "uncertain_seed1_max_20percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_20_percent/result_folder": "uncertain_seed20_max_20percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_20_percent/result_folder": "uncertain_seed42_max_20percent",}
paths_uncertain_max30 = {"/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_30_percent/result_folder": "uncertain_seed1_max_30percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_30_percent/result_folder": "uncertain_seed20_max_30percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_30_percent/result_folder": "uncertain_seed42_max_30percent",}
paths_uncertain_max50 = {"/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed1/max_50_percent/result_folder": "uncertain_seed1_max_50percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed20/max_50_percent/result_folder": "uncertain_seed20_max_50percent",
         "/mnt/volume/shared/new_results/mixed_uncertain_fixbug_seed42/max_50_percent/result_folder": "uncertain_seed42_max_50percent",}
all_paths = {"exp0": paths_exp0,
             "exp3": paths_exp3,
             "mix10": paths_mix10,
             "mix20": paths_mix20,
             "mix30": paths_mix30,
             "mix50": paths_mix50,
             "uncertain_max10": paths_uncertain_max10,
             "uncertain_max20": paths_uncertain_max20,
             "uncertain_max30": paths_uncertain_max30,
             "uncertain_max50": paths_uncertain_max50,
            }

In [15]:
res_all = pd.DataFrame(columns=['path', 'acc', 'f1', 'pr', 're', 'auc_pr_RUNX1_RUNX1T1',
       'auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11',
       'auc_roc_RUNX1_RUNX1T1', 'auc_roc_control', 'auc_roc_NPM1',
       'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11', 'c_acc_RUNX1_RUNX1T1',
       'c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11',
       'c_f1_RUNX1_RUNX1T1', 'c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA',
       'c_f1_CBFB_MYH11'])
res_mean_all = pd.DataFrame(columns=['path', 'acc', 'f1', 'pr', 're', 'auc_pr_RUNX1_RUNX1T1',
       'auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11',
       'auc_roc_RUNX1_RUNX1T1', 'auc_roc_control', 'auc_roc_NPM1',
       'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11', 'c_acc_RUNX1_RUNX1T1',
       'c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11',
       'c_f1_RUNX1_RUNX1T1', 'c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA',
       'c_f1_CBFB_MYH11'])
res_std_all = pd.DataFrame(columns=['path', 'acc', 'f1', 'pr', 're', 'auc_pr_RUNX1_RUNX1T1',
       'auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11',
       'auc_roc_RUNX1_RUNX1T1', 'auc_roc_control', 'auc_roc_NPM1',
       'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11', 'c_acc_RUNX1_RUNX1T1',
       'c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11',
       'c_f1_RUNX1_RUNX1T1', 'c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA',
       'c_f1_CBFB_MYH11'])
for key in all_paths.keys():
    paths = all_paths[key]
    df = get_res_df(paths)
    res = get_mean_std(df)
    res.to_csv(f"./results/{key}.csv")
    res = res.iloc[-2:]
    res.at["mean", "path"] = f"mean_{key}"
    res.at["std", "path"] = f"std_{key}"
    res_std = res.iloc[-1:]
    res_mean = res.iloc[:1]
    res_all = pd.concat([res_all, res])
    res_mean_all = pd.concat([res_mean_all, res_mean])
    res_std_all = pd.concat([res_std_all, res_std])
res_all

               precision    recall  f1-score   support

RUNX1_RUNX1T1       1.00      0.67      0.80         6
      control       1.00      1.00      1.00        12
         NPM1       0.86      0.86      0.86         7
     PML_RARA       1.00      1.00      1.00         5
   CBFB_MYH11       0.67      0.86      0.75         7

     accuracy                           0.89        37
    macro avg       0.90      0.88      0.88        37
 weighted avg       0.91      0.89      0.89        37

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.80      0.67      0.73         6
      control       1.00      1.00      1.00        12
         NPM1       0.86      0.86      0.86         7
     PML_RARA       1.00      0.80      0.89         5
   CBFB_MYH11       0.67      0.86      0.75         7

     accuracy                           0.86        37
    macro avg       0.86      0.84      0.84        37
 weighted avg       0.88      0.86      0.87        37

    

/tmp/ipykernel_440638/3801319239.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_all = pd.concat([res_all, res])
/tmp/ipykernel_440638/3801319239.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_mean_all = pd.concat([res_mean_all, res_mean])
/tmp/ipykernel_440638/3801319239.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result d

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.50      0.50      0.50         6
      control       1.00      1.00      1.00        12
         NPM1       0.50      0.57      0.53         7
     PML_RARA       1.00      1.00      1.00         5
   CBFB_MYH11       0.83      0.71      0.77         7

     accuracy                           0.78        37
    macro avg       0.77      0.76      0.76        37
 weighted avg       0.79      0.78      0.79        37

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.00      0.00      0.00         6
      control       1.00      1.00      1.00        12
         NPM1       0.38      0.71      0.50         7
     PML_RARA       1.00      0.80      0.89         5
   CBFB_MYH11       0.83      0.71      0.77         7

     accuracy                           0.70        37
    macro avg       0.64      0.65      0.63        37
 weighted avg       0.69      0.70      0.68        37

    

,path,acc,f1,pr,re,auc_pr_RUNX1_RUNX1T1,auc_pr_control,auc_pr_NPM1,auc_pr_PML_RARA,auc_pr_CBFB_MYH11,...,c_acc_RUNX1_RUNX1T1,c_acc_control,c_acc_NPM1,c_acc_PML_RARA,c_acc_CBFB_MYH11,c_f1_RUNX1_RUNX1T1,c_f1_control,c_f1_NPM1,c_f1_PML_RARA,c_f1_CBFB_MYH11
mean,mean_exp0,0.864865,0.864865,0.864865,0.864865,0.886431,1.000000e+00,0.875747,1.000000,0.818223,...,0.611111,1.0,0.857143,8.666667e-01,0.857143,0.709091,1.000000,0.838095,0.925926,7.500000e-01
std,std_exp0,0.027027,0.027027,0.027027,0.027027,0.033711,0.000000e+00,0.008855,0.000000,0.004436,...,0.096225,0.0,0.000000,1.154701e-01,0.000000,0.101232,0.000000,0.032991,0.064150,0.000000e+00
mean,mean_exp3,0.756757,0.756757,0.756757,0.756757,0.464833,1.000000e+00,0.668949,1.000000,0.867659,...,0.333333,1.0,0.619048,8.666667e-01,0.761905,0.333333,0.986667,0.549573,0.925926,7.794872e-01
std,std_exp3,0.046812,0.046812,0.046812,0.046812,0.062008,0.000000e+00,0.047774,0.000000,0.060805,...,0.288675,0.0,0.082479,1.154701e-01,0.082479,0.288675,0.023094,0.059382,0.064150,1.776462e-02
mean,mean_mix10,0.900901,0.900901,0.900901,0.900901,0.958604,1.000000e+00,0.916357,0.987778,0.874556,...,0.888889,1.0,0.857143,8.000000e-01,0.857143,0.888889,1.000000,0.838095,0.888889,8.190476e-01
std,std_mix10,0.015604,0.015604,0.015604,0.015604,0.039872,7.850462e-17,0.025625,0.021170,0.107020,...,0.096225,0.0,0.000000,1.359740e-16,0.000000,0.096225,0.000000,0.032991,0.000000,3.299144e-02
mean,mean_mix20,0.918919,0.918919,0.918919,0.918919,0.985119,1.000000e+00,0.893354,1.000000,0.829186,...,0.944444,1.0,0.809524,8.666667e-01,0.904762,0.897436,1.000000,0.849206,0.925926,8.630952e-01
std,std_mix20,0.027027,0.027027,0.027027,0.027027,0.025775,0.000000e+00,0.020778,0.000000,0.039264,...,0.096225,0.0,0.082479,1.154701e-01,0.082479,0.117502,0.000000,0.013746,0.064150,1.030983e-02
mean,mean_mix30,0.864865,0.864865,0.864865,0.864865,0.952177,1.000000e+00,0.916072,1.000000,0.857607,...,0.666667,1.0,0.761905,9.333333e-01,0.857143,0.775758,1.000000,0.798535,0.962963,7.205882e-01
std,std_mix30,0.000000,0.000000,0.000000,0.000000,0.061854,0.000000e+00,0.007077,0.000000,0.000482,...,0.000000,0.0,0.082479,1.154701e-01,0.000000,0.041989,0.000000,0.050756,0.064150,2.547134e-02


In [16]:
res_all.to_csv("./results/all_results.csv")
res_mean_all.to_csv("./results/mean_all_results.csv")
res_std_all.to_csv("./results/std_all_results.csv")

In [83]:
df = res.iloc[-2:]
df.at["mean", "path"] = "mean"
df.at["std", "path"] = "std"
df

,path,acc,f1,pr,re,auc_pr_RUNX1_RUNX1T1,auc_pr_control,auc_pr_NPM1,auc_pr_PML_RARA,auc_pr_CBFB_MYH11,...,c_acc_RUNX1_RUNX1T1,c_acc_control,c_acc_NPM1,c_acc_PML_RARA,c_acc_CBFB_MYH11,c_f1_RUNX1_RUNX1T1,c_f1_control,c_f1_NPM1,c_f1_PML_RARA,c_f1_CBFB_MYH11
mean,mean,0.900901,0.900901,0.900901,0.900901,0.925132,1.0,0.886264,1.0,0.865864,...,0.888889,1.0,0.714286,1.0,0.857143,0.863248,1.0,0.750916,1.0,0.838095
std,std,0.015604,0.015604,0.015604,0.015604,0.060498,0.0,0.063800,0.0,0.063882,...,0.096225,0.0,0.000000,0.0,0.000000,0.051813,0.0,0.031723,0.0,0.032991


In [87]:
res.columns

Index(['path', 'acc', 'f1', 'pr', 're', 'auc_pr_RUNX1_RUNX1T1',
       'auc_pr_control', 'auc_pr_NPM1', 'auc_pr_PML_RARA', 'auc_pr_CBFB_MYH11',
       'auc_roc_RUNX1_RUNX1T1', 'auc_roc_control', 'auc_roc_NPM1',
       'auc_roc_PML_RARA', 'auc_roc_CBFB_MYH11', 'c_acc_RUNX1_RUNX1T1',
       'c_acc_control', 'c_acc_NPM1', 'c_acc_PML_RARA', 'c_acc_CBFB_MYH11',
       'c_f1_RUNX1_RUNX1T1', 'c_f1_control', 'c_f1_NPM1', 'c_f1_PML_RARA',
       'c_f1_CBFB_MYH11'],
      dtype='object')